In [182]:
import pandas as pd
import numpy as np
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import multiprocessing as mp
from tqdm import tqdm
import pickle
import parmap
import re

In [183]:
from gensim import models, corpora
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation
from gensim.parsing.preprocessing import remove_stopwords, stem_text, strip_non_alphanum, strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_short, strip_numeric, stem_text
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [184]:
vote_file = pd.read_csv('brexit_full_updated3.csv')
vote_file

,Constituency ID,full name,PANO,Constituency Name,Party abbreviation,conShare,conShare2,conShare3,bill161,leave,party_old,ref,ref_dummy,party,old_party
0,E14000530,Gerald Howarth,7,Aldershot,Con,50.59,73.403947,73.403947,1.0,1.0,1.0,0.578978,1,1.0,1.0
1,E14000531,Wendy Morton,8,Aldridge-Brownhills,Con,52.05,69.940876,69.940876,1.0,0.0,1.0,0.677963,1,0.0,0.0
2,E14000532,Graham Brady,9,Altrincham and Sale West,Con,52.99,66.503514,66.503514,1.0,1.0,1.0,0.385878,0,1.0,1.0
3,E14000533,Nigel Mills,11,Amber Valley,Con,43.98,55.840528,55.840528,1.0,1.0,1.0,0.652991,1,1.0,1.0
4,E14000534,Nick Herbert,18,Arundel and South Downs,Con,60.79,84.442284,84.442284,NaN,0.0,1.0,0.497011,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,W07000076,Wayne David,114,Caerphilly,Lab,16.59,27.223499,27.223499,1.0,0.0,0.0,0.551360,1,0.0,NaN
656,W07000077,Chris Evans,336,Islwyn,Lab,15.16,23.639482,23.639482,1.0,0.0,0.0,0.589399,1,0.0,NaN
657,W07000078,Alun Cairns,589,Vale Of Glamorgan,Con,46.02,58.527280,58.527280,1.0,0.0,1.0,0.525507,1,0.0,0.0
658,W07000079,Kevin Brennan,129,Cardiff West,Lab,25.15,38.221884,38.221884,0.0,0.0,0.0,0.438226,0,0.0,NaN


In [185]:
politicians_party = {}
politicians = vote_file['full name'].unique()
for index, row in vote_file.iterrows():
    politicians_party[row['full name']] = row['party']

In [186]:
def preprocess(topic):
    ret = []
    topic = topic.strip()
    
    if '|' in topic:
        topics = topic.split('|')
        t = topics[0]
        t = t.strip()
        return t
        
    return topic

In [215]:
df1 = pd.read_csv('2015_commons.csv')
df2 = pd.read_csv('2016_commons.csv')
df = pd.concat([df1, df2])
df = df.drop(['Government'], axis=1)
df = df.loc[(df['Party'] == 'Conservative') | (df['Party'] == 'Labour')]
df = df.loc[df['Name'].isin(politicians)]
df['Date'] = pd.to_datetime(df['Date'])
mask = (df['Date'] > '2015-5-1') & (df['Date'] <= '2016-6-30')
df = df.loc[mask]
df['party2'] = df['Name'].apply(lambda x: politicians_party[x])
df['Topic'] = df['Topic'].apply(lambda x: preprocess(x))
print(df['Topic'].unique())
# df = df.loc[df['Topic'] == 'Parliament, government and politics']
# df = df.loc[df['Topic'] == 'European Union']
df = df.loc[df['Topic'] == 'Crime, civil law, justice and rights']
# df = df.loc[df['Topic'] == 'Health services and medicine']
# df = df.loc[df['Topic'] == 'International affairs']
cols = df.columns
cols = ['date', 'speaker', 'name', 'Party', 'Topic', 'transcript', 'party2']
df.columns = cols
df = df.drop(['speaker'], axis=1)
df.head()

['admin' 'Employment and training' 'Others'
 'Parliament, government and politics'
 'Crime, civil law, justice and rights' 'Defence' 'International affairs'
 'Agriculture, animals, food and rural affairs'
 'Health services and medicine' 'Communities and families'
 'Culture, media and sport' 'Economy and finance'
 'Asylum, immigration and nationality' 'European Union' 'Education'
 'Housing and planning' 'Energy and environment' 'Transport'
 'Business, industry and consumers' 'Science and technology'
 'Social security and pensions' 'Social services']


,date,name,Party,Topic,transcript,party2
18826,2015-05-28,Theresa May,Conservative,"Crime, civil law, justice and rights",The Gracious Speech we heard yesterday set out...,0.0
18827,2015-05-28,Jonathan Ashworth,Labour,"Crime, civil law, justice and rights",Estimates provided by the House of Commons Lib...,0.0
18828,2015-05-28,Theresa May,Conservative,"Crime, civil law, justice and rights","When we first came to office, we made it clear...",0.0
18829,2015-05-28,Kate Green,Labour,"Crime, civil law, justice and rights",The Home Secretary may be aware of two separat...,0.0
18830,2015-05-28,Theresa May,Conservative,"Crime, civil law, justice and rights",We have done a significant amount in relation ...,0.0


In [216]:
df['transcript'] = df[['name','transcript']].groupby(['name'])['transcript'].transform(lambda x: ','.join(x))

In [217]:
df = df.drop_duplicates()

In [218]:
df

,date,name,Party,Topic,transcript,party2
18826,2015-05-28,Theresa May,Conservative,"Crime, civil law, justice and rights",The Gracious Speech we heard yesterday set out...,0.0
18827,2015-05-28,Jonathan Ashworth,Labour,"Crime, civil law, justice and rights",Estimates provided by the House of Commons Lib...,0.0
18829,2015-05-28,Kate Green,Labour,"Crime, civil law, justice and rights",The Home Secretary may be aware of two separat...,0.0
18831,2015-05-28,Yvette Cooper,Labour,"Crime, civil law, justice and rights",The Home Secretary talked about the cuts that ...,0.0
18833,2015-05-28,Andrew Gwynne,Labour,"Crime, civil law, justice and rights",But of course it is not just about the numbers...,0.0
...,...,...,...,...,...,...
35629,2016-06-29,Stephen Doughty,Labour,"Crime, civil law, justice and rights","First, I welcome the new Under-Secretary of St...",0.0
35633,2016-06-29,Andy Slaughter,Labour,"Crime, civil law, justice and rights","We are a little ahead of time, but I will brea...",0.0
35635,2016-06-29,Andrew Gwynne,Labour,"Crime, civil law, justice and rights",But of course it is not just about the numbers...,0.0
35637,2016-06-29,Louise Haigh,Labour,"Crime, civil law, justice and rights",Does the right hon. Lady agree that in order t...,0.0


In [219]:
def get_party(row):
    if row['Party'] == 'Conservative' and row['party2'] == 1:
        party = 'leave_conservative'
    elif row['Party'] == 'Labour' and row['party2'] == 0:
        party = 'remain_labour'
    elif row['Party'] == 'Labour' and row['party2'] == 1:
        party = 'leave_labour'
    else:
        party = 'remain_conservative'
        
    return party

df['party'] = df.apply(lambda x: get_party(x), axis=1)

In [220]:
df

,date,name,Party,Topic,transcript,party2,party
18826,2015-05-28,Theresa May,Conservative,"Crime, civil law, justice and rights",The Gracious Speech we heard yesterday set out...,0.0,remain_conservative
18827,2015-05-28,Jonathan Ashworth,Labour,"Crime, civil law, justice and rights",Estimates provided by the House of Commons Lib...,0.0,remain_labour
18829,2015-05-28,Kate Green,Labour,"Crime, civil law, justice and rights",The Home Secretary may be aware of two separat...,0.0,remain_labour
18831,2015-05-28,Yvette Cooper,Labour,"Crime, civil law, justice and rights",The Home Secretary talked about the cuts that ...,0.0,remain_labour
18833,2015-05-28,Andrew Gwynne,Labour,"Crime, civil law, justice and rights",But of course it is not just about the numbers...,0.0,remain_labour
...,...,...,...,...,...,...,...
35629,2016-06-29,Stephen Doughty,Labour,"Crime, civil law, justice and rights","First, I welcome the new Under-Secretary of St...",0.0,remain_labour
35633,2016-06-29,Andy Slaughter,Labour,"Crime, civil law, justice and rights","We are a little ahead of time, but I will brea...",0.0,remain_labour
35635,2016-06-29,Andrew Gwynne,Labour,"Crime, civil law, justice and rights",But of course it is not just about the numbers...,0.0,remain_labour
35637,2016-06-29,Louise Haigh,Labour,"Crime, civil law, justice and rights",Does the right hon. Lady agree that in order t...,0.0,remain_labour


In [221]:
df['party'].unique()

array(['remain_conservative', 'remain_labour', 'leave_labour',
       'leave_conservative'], dtype=object)

In [222]:
def strip_short2(text):
    return strip_short(text, minsize=4)


def preprocess_text(text):
    FILTERS = [lambda x: x.lower(), strip_multiple_whitespaces, strip_tags, strip_punctuation,
                   strip_non_alphanum, strip_numeric, strip_short2]
    return preprocess_string(text, FILTERS)

In [15]:
X = df['transcript'].values

In [18]:
preprocessed_X = parmap.map(preprocess_text, X, pm_pbar=True)
tagged_X = [models.doc2vec.TaggedDocument(doc, [i]) for i, doc in enumerate(preprocessed_X)]

20608it [01:54, 180.76it/s]                           


In [20]:
# print('training doc2vec')
# doc2vec_model = Doc2Vec(vector_size=300, window=3, workers=mp.cpu_count(), epochs=40)
# doc2vec_model.build_vocab(tagged_X)
# doc2vec_model.train(tagged_X, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)

training doc2vec


In [21]:
doc2vec_model.save('./text_split_doc2vec')

In [197]:
doc2vec_model = Doc2Vec.load('./text_split_doc2vec')

In [223]:
X = []
y = []

In [224]:
df_remain_labour = df.loc[df['party'] == 'remain_labour']
df_remain_labour

,date,name,Party,Topic,transcript,party2,party
18827,2015-05-28,Jonathan Ashworth,Labour,"Crime, civil law, justice and rights",Estimates provided by the House of Commons Lib...,0.0,remain_labour
18829,2015-05-28,Kate Green,Labour,"Crime, civil law, justice and rights",The Home Secretary may be aware of two separat...,0.0,remain_labour
18831,2015-05-28,Yvette Cooper,Labour,"Crime, civil law, justice and rights",The Home Secretary talked about the cuts that ...,0.0,remain_labour
18833,2015-05-28,Andrew Gwynne,Labour,"Crime, civil law, justice and rights",But of course it is not just about the numbers...,0.0,remain_labour
18835,2015-05-28,Clive Efford,Labour,"Crime, civil law, justice and rights",The Home Secretary just said that the Metropol...,0.0,remain_labour
...,...,...,...,...,...,...,...
35629,2016-06-29,Stephen Doughty,Labour,"Crime, civil law, justice and rights","First, I welcome the new Under-Secretary of St...",0.0,remain_labour
35633,2016-06-29,Andy Slaughter,Labour,"Crime, civil law, justice and rights","We are a little ahead of time, but I will brea...",0.0,remain_labour
35635,2016-06-29,Andrew Gwynne,Labour,"Crime, civil law, justice and rights",But of course it is not just about the numbers...,0.0,remain_labour
35637,2016-06-29,Louise Haigh,Labour,"Crime, civil law, justice and rights",Does the right hon. Lady agree that in order t...,0.0,remain_labour


In [225]:
for index in range(len(df_remain_labour)):
    row = df_remain_labour.iloc[index]
    X.append(row['transcript'])
    y.append(0)

In [226]:
df_leave_conservative = df.loc[df['party'] == 'leave_conservative']
df_leave_conservative

,date,name,Party,Topic,transcript,party2,party
18843,2015-05-28,Rehman Chishti,Conservative,"Crime, civil law, justice and rights",Of course freedom of speech is a basic right t...,1.0,leave_conservative
18855,2015-05-28,Richard Fuller,Conservative,"Crime, civil law, justice and rights",My right hon. Friend is right to point out how...,1.0,leave_conservative
18864,2015-05-28,Robert Syms,Conservative,"Crime, civil law, justice and rights",Given the right hon. Lady’s criticism of the G...,1.0,leave_conservative
18881,2015-05-28,Liam Fox,Conservative,"Crime, civil law, justice and rights",The Gracious Speech sets out a programme with ...,1.0,leave_conservative
18888,2015-05-28,Andrew Stephenson,Conservative,"Crime, civil law, justice and rights",Let me begin by congratulating the hon. and le...,1.0,leave_conservative
...,...,...,...,...,...,...,...
33781,2016-06-14,Bob Blackman,Conservative,"Crime, civil law, justice and rights","Technology moves on apace, and criminals are v...",1.0,leave_conservative
33790,2016-06-14,Maria Caulfield,Conservative,"Crime, civil law, justice and rights",17. What steps the Government are taking to e...,1.0,leave_conservative
33831,2016-06-14,Derek Thomas,Conservative,"Crime, civil law, justice and rights","T5. What support is, or will be, available t...",1.0,leave_conservative
33843,2016-06-14,Christopher Chope,Conservative,"Crime, civil law, justice and rights",Does my hon. Friend believe that we can do mor...,1.0,leave_conservative


In [227]:
for index in range(len(df_leave_conservative)):
    row = df_leave_conservative.iloc[index]
    X.append(row['transcript'])
    y.append(1)

In [228]:
y = np.array(y)
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
input_Y = y.reshape(-1,1)
enc.fit(input_Y)
input_Y = enc.transform(input_Y).toarray()

In [229]:
from sklearn.utils import shuffle
X, y = shuffle(X, y, random_state=0)

In [230]:
preprocessed_X = parmap.map(preprocess_text, X, pm_pbar=True)
tagged_X = [models.doc2vec.TaggedDocument(doc, [i]) for i, doc in enumerate(preprocessed_X)]

1134it [00:01, 623.85it/s]              


In [231]:
print('preparing inputs')

def get_doc_vec(doc):
    return doc2vec_model.infer_vector(doc.words)

inputs = parmap.map(get_doc_vec, tagged_X, pm_pbar=True)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs, input_Y, stratify=y, test_size=0.3, random_state=42)
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)
print('X_train: ', X_train.shape)
print('y_train: ', y_train.shape)
print('X_test: ', X_test.shape)
print('y_test: ', y_test.shape)

preparing inputs


1134it [00:09, 119.54it/s]                          


X_train:  (788, 300)
y_train:  (788, 2)
X_test:  (339, 300)
y_test:  (339, 2)


In [232]:
import tensorflow as tf

In [233]:
def build_network():
    model = tf.keras.models.Sequential([
        tf.keras.Input(shape=(300)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
    ])
    loss_fn =  tf.keras.losses.BinaryCrossentropy (from_logits=False, name='binary_crossentropy')

    model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=loss_fn,
              metrics=['accuracy'])
    
    return model

In [234]:
model = build_network()
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
# class_weights = class_weight.compute_class_weight('balanced', np.unique(Y), Y)
model.fit(X_train, y_train, batch_size=8, epochs=200, callbacks=[callback],
          validation_data=(X_test, y_test))

Epoch 1/200
99/99 [==============================] - 1s 5ms/step - loss: 0.6967 - accuracy: 0.5914 - val_loss: 0.6766 - val_accuracy: 0.5929
Epoch 2/200
99/99 [==============================] - 0s 3ms/step - loss: 0.6601 - accuracy: 0.6218 - val_loss: 0.6757 - val_accuracy: 0.5870
Epoch 3/200
99/99 [==============================] - 0s 3ms/step - loss: 0.6385 - accuracy: 0.6409 - val_loss: 0.6953 - val_accuracy: 0.5900
Epoch 4/200
99/99 [==============================] - 0s 3ms/step - loss: 0.6183 - accuracy: 0.6497 - val_loss: 0.7132 - val_accuracy: 0.5782
Epoch 5/200
99/99 [==============================] - 0s 3ms/step - loss: 0.6108 - accuracy: 0.6472 - val_loss: 0.7084 - val_accuracy: 0.5900
Epoch 6/200
99/99 [==============================] - 0s 3ms/step - loss: 0.5960 - accuracy: 0.6574 - val_loss: 0.7501 - val_accuracy: 0.5811
Epoch 7/200
99/99 [==============================] - 0s 3ms/step - loss: 0.5841 - accuracy: 0.6561 - val_loss: 0.7471 - val_accuracy: 0.5929


In [235]:
test_df = pd.read_csv('test_key_speeches_combined.csv')
test_df

,text
0,We all agree that it is vital that everyone is...
1,"If we Vote Leave on 23 June, the Government wi..."
2,Thirteen Government ministers and senior Conse...
3,Michael Gove and Dominic Raab have today made ...
4,"Michael Gove, Boris Johnson, Gisela Stuart and..."
5,"Michael Gove, Boris Johnson, and Gisela Stuart..."
6,A STRONGER NHS AND MORE MONEY FOR THOSE IN NEE...
7,Liam Fox MP delivered a speech today at Vote L...
8,"Statement by Michael Gove, Boris Johnson, Prit..."
9,"On 23 June, Sun readers can decide whether we ..."


In [236]:
def predict(X):
    preprocessed_X = preprocess_text(X)
    vec_X = doc2vec_model.infer_vector(preprocessed_X)
    vec_X = vec_X.reshape(1, vec_X.shape[0])
    pred = model.predict(vec_X)
    return pred

In [237]:
predict(test_df.iloc[0]['text'])

array([[0.7780544 , 0.22194555]], dtype=float32)

In [238]:
rows = []

n = len(test_df)
for index in tqdm(range(n)):
    row = test_df.iloc[index]
    transcript = row['text']
    pred = predict(transcript)[0]
    rows.append([transcript, pred[0], pred[1]])
    
aff_df = pd.DataFrame(rows, columns=['transcript', 'remain_labour', 'leave_conservative'])
aff_df

100%|██████████| 24/24 [00:04<00:00,  5.32it/s]


,transcript,remain_labour,leave_conservative
0,We all agree that it is vital that everyone is...,0.657489,0.342511
1,"If we Vote Leave on 23 June, the Government wi...",0.877851,0.122149
2,Thirteen Government ministers and senior Conse...,0.564983,0.435017
3,Michael Gove and Dominic Raab have today made ...,0.902217,0.097783
4,"Michael Gove, Boris Johnson, Gisela Stuart and...",0.887516,0.112484
5,"Michael Gove, Boris Johnson, and Gisela Stuart...",0.923643,0.076357
6,A STRONGER NHS AND MORE MONEY FOR THOSE IN NEE...,0.863009,0.136991
7,Liam Fox MP delivered a speech today at Vote L...,0.874679,0.125321
8,"Statement by Michael Gove, Boris Johnson, Prit...",0.935163,0.064837
9,"On 23 June, Sun readers can decide whether we ...",0.988461,0.011539


In [239]:
aff_df.to_csv('key_speeches_predictions.csv', index=False)